# Compare results against larch

We use estimation data as prepared by larch and draw the Asim way and our way, then compare to larch probabilities.


In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [10]:
import os
import argparse

import numpy as np
import pandas as pd

from activitysim.cli import run
from activitysim.core import inject

from fru_utils import mode_choice_for_trip

pd.set_option("max_columns", 500)

In [5]:
root_dir = "/mnt/c/Users/jan.zill/code/activitysim"
example_dir = os.path.join(root_dir, "test_example_sf")
os.chdir(example_dir)

In [7]:
#!activitysim run -c configs -o output -d data_sf

In [8]:
parser = argparse.ArgumentParser()
run.add_run_args(parser)
args = parser.parse_args(['-c', 'configs', '-o', 'output', '-d', 'data_sf'])
if not inject.is_injectable('preload_injectables'):
    from activitysim import abm  # register abm steps and other abm-specific injectables
run.handle_standard_args(args)  # possibly update injectables

In [68]:
larch_chooser_data = pd.read_csv(os.path.join(root_dir, "test_estimation_sf", "choosers_larch.csv"))
larch_probs = pd.read_csv(os.path.join(root_dir, "test_estimation_sf", "probabilities_larch.csv"), index_col=0)
# __caseids__ are tour ids, with trips in same order
assert (larch_probs.index.values ==  larch_chooser_data.tour_id).all()
larch_probs.index = larch_chooser_data.trip_id

In [50]:
larch_tours = pd.read_csv(os.path.join(root_dir, "test_estimation_sf", "output", "estimation_data_bundle",
                                       "trip_mode_choice", "trip_mode_choice_values_combined.csv"))
trips_merged_cols = ['person_id', 'household_id', 'primary_purpose', 'trip_num', 'outbound',
                     'trip_count', 'destination', 'origin', 'tour_id', 'purpose',
                     'destination_logsum', 'depart', 'hhsize', 'age', 'auto_ownership',
                     'number_of_participants', 'tour_category', 'parent_tour_id',
                     'tour_mode', 'duration', 'value_of_time', 'tour_type',
                     'free_parking_at_work', 'trip_period', "trip_id"]

larch_trips_merged = larch_tours[trips_merged_cols].set_index("trip_id")

In [87]:
def compare_mode_share_simulation(trip_id_to_check, num_samples, larch_trips_merged, larch_probs):
    t_, c_, ns_ = mode_choice_for_trip(choose_individual_max_utility=True, trip_id_to_check=trip_id_to_check,
                                       num_samples=num_samples, trips_merged=larch_trips_merged)
    sim_mode_shares = c_.trip_mode.value_counts() / c_.shape[0]

    t_asim, c_asim, _ = mode_choice_for_trip(choose_individual_max_utility=False, trip_id_to_check=trip_id_to_check,
                                             num_samples=num_samples, trips_merged=larch_trips_merged)
    asim_mode_shares = c_asim.trip_mode.value_counts() / c_asim.shape[0]

    larch_mode_shares = larch_probs.loc[larch_probs.index == trip_id_to_check].T
    larch_mode_shares.columns = ["larch"]

    comp = sim_mode_shares.to_frame("me")\
        .merge(asim_mode_shares.to_frame("asim"), left_index=True, right_index=True,how="outer")\
        .merge(larch_mode_shares,left_index=True, right_index=True, how="outer").fillna(0)

    return comp

In [124]:
larch_probs

,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
trip_id,,,,,,,,,,,,,,,,,,,,,
23695361,0.000000,0.0,2.027363e-06,0.0,1.061530e-06,0.0,0.111823,0.0,0.263130,0.625043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.489336e-08,4.965881e-08,4.072015e-07
23695365,0.000000,0.0,2.943297e-09,0.0,1.967559e-08,0.0,0.119603,0.0,0.272258,0.608139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.435815e-08,4.996681e-08,4.311721e-07
23760961,0.999664,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000010,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.724941e-05,1.754653e-04,1.035425e-04
23760962,0.998934,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000579,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.066593e-04,2.780937e-04,1.021060e-04
23760963,0.997988,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.001509,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.039326e-04,2.850044e-04,1.143830e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464724829,0.000000,0.0,3.268925e-05,0.0,3.022226e-05,0.0,0.691838,0.0,0.308044,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.606576e-05,4.836729e-07,1.874068e-05
2464847169,0.000000,0.0,2.321209e-03,0.0,1.213314e-03,0.0,0.249261,0.0,0.745641,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.274883e-04,5.407950e-04,5.947334e-04
2464847173,0.000000,0.0,1.269812e-03,0.0,7.535248e-04,0.0,0.186394,0.0,0.810411,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.063342e-04,4.098183e-04,4.563525e-04


In [144]:
%%time

trip_id_to_check = 24106017 # obvious results with 1e4 samples: 23695361 24106017
# better with 1e5 samples: 1932588125
# requires higher precision, check when time: 38280865
num_samples = 100000  # 1e5 takes about 80s

comp = compare_mode_share_simulation(trip_id_to_check, num_samples, larch_trips_merged, larch_probs)
comp.loc[(comp > 1e-3).any(axis=1), :].applymap(lambda x: f"{x:.3%}")

estimation bundle trip_mode_choice not in settings file estimation.yaml


trip_mode_choice tour_type 'work' (100000 trips)
Done


estimation bundle trip_mode_choice not in settings file estimation.yaml


trip_mode_choice tour_type 'work' (100000 trips)
Done
CPU times: user 1min 10s, sys: 3.98 s, total: 1min 14s
Wall time: 1min 9s


,me,asim,larch
SHARED2FREE,0.395%,0.420%,0.359%
SHARED3FREE,0.214%,0.226%,0.188%
TNC_SINGLE,0.212%,0.231%,0.202%
WALK,0.488%,0.466%,0.435%
WALK_HVY,69.439%,69.422%,60.165%
WALK_LOC,9.697%,9.596%,14.513%
WALK_LRF,19.390%,19.469%,23.977%


In [139]:
#comp.loc[(comp > 0).any(axis=1), :]

In [140]:
0.72 * 0.35

0.252

In [ ]:
# check what happens if you multiply them - like Train at the lower level? but then 0<scale<1 for both